In [ ]:
import pickle
import os

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [ ]:
plot_dir = "../../../plots/XHY/Apr12"
_ = os.system(f"mkdir -p {plot_dir}")

In [ ]:
year = "2017"

with open(f"/eos/uscms/store/user/rkansal/bbVV/xhy/Apr4/{year}_combined.pkl", "rb") as f:
    out = pickle.load(f)

In [ ]:
def mxmy(sample):
    mY = int(sample.split("-")[-1])
    mX = int(sample.split("NMSSM_XToYHTo2W2BTo4Q2B_MX-")[1].split("_")[0])

    return mX, mY

In [ ]:
out[year]["NMSSM_XToYHTo2W2BTo4Q2B_MX-1000_MY-100"]["2017"]

In [ ]:
acceptances = np.array(
    [
        [
            *mxmy(s),
            sdict[year][s]["cutflow"]["ak8bb_txbb"] / sdict[year][s]["cutflow"]["has_4q"],
        ]
        for s, sdict in out[year].items()
    ]
)

fraction_4qs = np.array(
    [
        [
            *mxmy(s),
            sdict[year][s]["h"].values()[4] / sdict[year][s]["h"].sum(flow=True),
        ]
        for s, sdict in out[year].items()
    ]
)

fraction_3q4qs = np.array(
    [
        [
            *mxmy(s),
            sum(sdict[year][s]["h"].values()[3:]) / sdict[year][s]["h"].sum(flow=True),
        ]
        for s, sdict in out[year].items()
    ]
)

In [ ]:
def scatter2d(arr, title, name):
    fig, ax = plt.subplots(figsize=(14, 12))
    mappable = plt.scatter(arr[:, 1], arr[:, 0], s=150, c=arr[:, 2], cmap="turbo")
    plt.title(title)
    plt.xlabel(r"$m_Y$ (GeV)")
    plt.ylabel(r"$m_X$ (GeV)")
    plt.colorbar(mappable)
    plt.savefig(name, bbox_inches="tight")

In [ ]:
scatter2d(acceptances, "Preselection Acceptance", f"{plot_dir}/acceptance.pdf")

In [ ]:
scatter2d(fraction_4qs, "Fraction of Y(VV) jets containing 4 quarks", f"{plot_dir}/fraction_4q.pdf")

In [ ]:
f4q_gt50 = np.copy(fraction_4qs)
f4q_gt50[:, 2][f4q_gt50[:, 2] < 0.5] = 0
scatter2d(
    fraction_4qs[fraction_4qs[:, 2] >= 0.5],
    "Fraction of Y(VV) jets containing 4 quarks > 0.5",
    f"{plot_dir}/fraction_4q_gt50.pdf",
)

In [ ]:
scatter2d(
    fraction_3q4qs,
    "Fraction of Y(VV) jets containing 3 or 4 quarks",
    f"{plot_dir}/fraction_3q4q.pdf",
)

In [ ]:
target_samples = [
    f"NMSSM_XToYH_MX{int(mx)}_MY{int(my)}_HTo2bYTo2W_hadronicDecay"
    for mx, my, frac in fraction_4qs
    if (frac >= 0.5 and my >= 120)
]

In [ ]:
target_samples